### Importaciones

In [1]:
# Paquetes necesarios para la ejecución del notebook
import win32com.client
import os
import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl.workbook import Workbook
from copy import copy
from openpyxl.utils import get_column_letter, column_index_from_string
import polars as pl
import math
import time
import pygetwindow as gw
from msal import ConfidentialClientApplication
import requests
from PIL import Image

# Módulos personalizados
import update_source_report_files_library as usf
import update_transportista_resumen_files_library as utr
import make_automate_powerbi_report as pb_automate
import set_mail_report as smr

### Configuracion

In [2]:
# Constantes usadas en el notebook
MAPI = "MAPI" # Messaging Application Programming Interface
DOT = "."
OUTLOOK = "Outlook"
APPLICATION = "Application"
MAIL_ITEM_CODE = 43

# Diccionarios
outlook_folder_codes = {
    0: 'Calendario',
    1: 'Contactos',
    2: 'Borradores',
    3: 'Diario / Jornal',
    4: 'Notas',
    5: 'Tareas',
    6: 'Bandeja de entrada',
    7: 'Bandeja de salida',
    8: 'Elementos enviados',
    9: 'Elementos eliminados',
    10: 'Bandeja de correo del servidor',
    11: 'Conflictos',
    12: 'Elementos de sincronizacion local',
    13: 'Elementos de sincronizacion (Envio)',
    14: 'Elementos de sincronización (Recibo)',
    15: 'Elementos de sincronización completa',
    16: 'Diario de formularios',
    17: 'Carpeta de búsqueda',
    18: 'Bandeja para reglas cliente',
    19: 'Carpeta de sugerencias de correo',
}
parse_locaciones = {
    '06 AYA EL PEDREGAL': 'Pedregal',
    '38 AYA ATICO': 'Atico',
    '40 AYA CHALA': 'Chala',
    '88 AYA CAMANA': 'Camana'
}
meta = {
    '06 AYA EL PEDREGAL': {
        2023: {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: 0.7, 6: 0.7, 7: 0.7, 8:0.7, 9:0.7, 10:0.7, 11:0.7, 12:0.7},
        2024: {1: 1.2, 2: 1.2, 3: 1.2, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1},
        2025: {1: 0.88, 2: 0.88, 3: 0.88, 4: 0.88, 5: 0.88, 6: 0.88, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0},
    },
    '38 AYA ATICO': {
        2023: {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: 0.7, 6: 0.7, 7: 0.7, 8:0.7, 9:0.7, 10:0.7, 11:0.7, 12:0.7},
        2024: {1: 0.4, 2: 0.4, 3: 0.4, 4: 0.48, 5: 0.48, 6: 0.48, 7: 0.48, 8: 0.48, 9: 0.48, 10: 0.48, 11: 0.48, 12: 0.48},
        2025: {1: 0.24, 2: 0.24, 3: 0.24, 4: 0.24, 5: 0.24, 6: 0.24, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0},
    },
    '40 AYA CHALA': {
        2023: {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: .7, 6: 0.7, 7: 0.7, 8:0.7, 9:0.7, 10:0.7, 11:0.7, 12:0.7},
        2024: {1: 0.5, 2: 0.5, 3: 0.5, 4: 0.60, 5: 0.60, 6: 0.60, 7: 0.60, 8: 0.60, 9: 0.60, 10: 0.60, 11: 0.60, 12: 0.60},
        2025: {1: 0.31, 2: 0.31, 3: 0.31, 4: 0.31, 5: 0.31, 6: 0.31, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0},
    },
    '88 AYA CAMANA': {
        2023: {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: 0.7, 6: 0.7, 7: 0.7, 8:0.7, 9:0.7, 10:0.7, 11:0.7, 12:0.7},
        2024: {1: 0.85, 2: 0.85, 3: 0.85, 4: 0.85, 5: 0.85, 6: 0.85, 7: 0.85, 8: 0.85, 9: 0.85, 10: 0.85, 11: 0.85, 12: 0.85},
        2025: {1: 0.57, 2: 0.57, 3: 0.57, 4: 0.57, 5: 0.57, 6: 0.57, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0},
    }
}
vendedores = {
    '06 AYA EL PEDREGAL': {
        '0000001013': 'AYALA CCAHUANA EDMUNDO',
        '0000001006': 'BARRAZA REVILLA FREDY',
        '0000001000': 'CARDENAS CHOQUECAJIA WHITNEY AWARD',
        '0000001001': 'CHARA ROJAS GISELA SOLEDAD',
        '0000001007': 'CHINO QUISPE JORGE LUIS',
        '0000006014': 'EDWAR QUISPE CHINO',
        '0000006009': 'GALLARDO HUAYLLA RENATO LEUTERIO',
        '0000006003': 'GALLEGOS NUÑEZ JUAN FREDY',
        '0000005999': 'GESTOR VIRTUAL',
        '0000001010': 'HUANCA MAMANI JACKSON',
        '0000006006': 'HUAYCHO TORRES ULBER',
        '0000006005': 'HUAYCHO TORRES ULBER',
        '0000001012': 'HUAYLLAZO HUACCHA CRISTHIAN PAOLO',
        '0000006012': 'NELSON RAUL CONZA CHARCA',
        '0000001002': 'PACHECO CONDORI ANDRES OSCAR',
        '0000001011': 'QUISPE CHINO WILIAN YURI',
        '0000001004': 'QUISPE HUAYLLA MARISOL S.',
        '0000006013': 'RONALD GONZALO HUILLCA MAMANI',
        '0000001003': 'TACO XESSPE KELY SOFIA',
        '0000001005': 'VALDERRAMA ELLIS JESSICA A.',
        '0000001008': 'VEND NVA RUTA 10',
        '0000001009': 'YUCRA JIMENEZ ANA LUZ',
        '0000001014': 'CHARA ROJAS GISELA SOLEDAD',
        '0000001015': 'BARRAZA REVILLA FREDY',
        '0000001019': 'BATALLANOS SANCA RODRIGO LEOPOLDO',
        '0000001016': 'PARI PUCHO FREDY OSWALDO',
        '0000001017': 'COAGUILLA MAMANI JOSE ALBERTO',
        '0000001018': 'ORTEGA MAMANI JORGE LUIS'
    },
    '38 AYA ATICO': {
        '0000001001': 'ALANYA RAMIREZ FERNANDO JOSUE',
        '0000005999': 'GESTOR VIRTUAL',
        '0000001000': 'SAUL ANDRES VIÑA VIZCARDO'
    },
    '40 AYA CHALA': {
        '0000001004': 'CANALES AGUILAR HILBERTO',
        '0000001007': 'CHIPANA JURADO JHORS EDUARDO',
        '0000006010': 'DIONICIO DANIEL HUARCAYA SALAZAR',
        '0000005999': 'GESTOR VIRTUAL',
        '0000006009': 'GLOBER FELIPE JARA MAQUER',
        '0000006005': 'GONZALES CHURA MAGNO ALFREDO',
        '0000001000': 'HERRERA TAPIA EDWIN DONATO',
        '0000006002': 'JARA CARAZAS RODOLFO JOSUE.L',
        '0000006007': 'JUAN CARLOS ARIAS BENITES',
        '0000001003': 'MAMANI TINTAYA KRISTHOFER',
        '0000001006': 'MAMANI TINTAYA KRISTHOFER',
        '0000001005': 'QUISE CCAPA EVER',
        '0000001002': 'RODRIGUEZ JOSE ANTONIO',
        '0000001001': 'VENDEDOR RT M1'        
    },
    '88 AYA CAMANA': {
        '0000001007': 'AUCAHUAQUI REVILLA DANIEL HITLER',
        '0000001011': 'CARAZAS REZA LUIS ALBERTO',
        '0000001003': 'CONDORCHOA SIERRA NEMECIO',
        '0000006001': 'CONDORCHOA SIERRA NEMESIO JESUS',
        '0000006002': 'DE LA CRUZ CALCINA ELAR',
        '0000006010': 'EDILBERTO RAMIREZ LARICO',
        '0000005999': 'GESTOR VIRTUAL',
        '0000001002': 'HUAMANI RODRIGUEZ YONATAN ANYIMZAN',
        '0000006011': 'JAIME CHAVEZ CONDORI',
        '0000001005': 'LLERENA DE LA CRUZ RICARDO SNEIDER',
        '0000001004': 'MEDINA VELASQUEZ JAVIER ENRIQUE',
        '0000006014': 'MOISES RICHARD CONDORCHOA SIERRA',
        '0000001008': 'MOLLO YUPANQUI JOSE OMAR',
        '0000001001': 'NO APLICA VENDEDOR',
        '0000006008': 'QUISPE CCACHUCO FREDY',
        '0000001010': 'RAMOS MAMANI RUBEN',
        '0000006016': 'RENATO ELEUTERIO GALLARADO HUAYLLA',
        '0000001006': 'SACARI CHOQUEHUANCA WILSON',
        '0000001000': 'SALAZAR HUAMANI SAUL ANTONIO',
        '0000001012': 'VEND RT X1',
        '0000001009': 'VIZCARDO BUSTAMANTE ALBERTH FRANCK'
    }
}

# Listas
locaciones = ['06 AYA EL PEDREGAL', '38 AYA ATICO', '40 AYA CHALA', '88 AYA CAMANA']

# Variables de configuracion
root_address = r'C:\Informacion\rechazos' # Direcccion de carpeta raiz
test_address = r'\prueba'
backup_address = r'\backup'
transportista = {
    "mail_subject": "Reporte de ordenes de carga diario",         # Nombre del asunto de correo
    "local_file_name": "Cf_programadas_por_transportista.csv",    # Nombre del archivo local
    "local_file_address": "",                                     # Direccion del archivo local
    "mail_file_address": "",                                      # Direccion del archivo del correo
    "mail_sheet_name": "Guias",
    "mail_received_time": "",
    "date": "Fecha",
    "date_format": "dd\/mm\/yyyy",
}
transportista_resumen = {
    "mail_subject": "Reporte de ordenes de carga diario",                 # Nombre del asunto de correo
    "local_file_name": "Cf_programadas_por_transportista_resumen.csv",    # Nombre del archivo local
    "local_file_address": "",                                             # Direccion del archivo local
    "date": "Fecha",
    "date_format": "dd\/mm\/yyyy",
}
ruta = {
    "mail_subject": "Venta perdida diaria por cliente y ruta - acum mes",   # Nombre del asunto de correo
    "local_file_name": "Cf_rech_por_ruta.csv",                              # Nombre del archivo local
    "local_file_address": "",                                               # Direccion del archivo local
    "mail_file_address": "",                                                # Direccion del archivo del correo
    "mail_sheet_name": "Motivos_VP (clte)",
    "mail_received_time": "",
    "date": "Día",
    "date_format": "d\/m\/yy",
}

<>:151: SyntaxWarning: invalid escape sequence '\/'
<>:158: SyntaxWarning: invalid escape sequence '\/'
<>:168: SyntaxWarning: invalid escape sequence '\/'
<>:151: SyntaxWarning: invalid escape sequence '\/'
<>:158: SyntaxWarning: invalid escape sequence '\/'
<>:168: SyntaxWarning: invalid escape sequence '\/'
C:\Users\AYACDA23\AppData\Local\Temp\ipykernel_17268\921609926.py:151: SyntaxWarning: invalid escape sequence '\/'
  "date_format": "dd\/mm\/yyyy",
C:\Users\AYACDA23\AppData\Local\Temp\ipykernel_17268\921609926.py:158: SyntaxWarning: invalid escape sequence '\/'
  "date_format": "dd\/mm\/yyyy",
C:\Users\AYACDA23\AppData\Local\Temp\ipykernel_17268\921609926.py:168: SyntaxWarning: invalid escape sequence '\/'
  "date_format": "d\/m\/yy",


### Obtener correos de outlook

In [ ]:
# Conectar a Outlook
#outlook_folder_code = int(input(f'{" ".join(["(" + str(key) + ": " + value + ")" for key, value in outlook_folder_codes.items()])}'))
outlook = win32com.client.Dispatch(OUTLOOK+DOT+APPLICATION).GetNamespace(MAPI)

outlook_folder = outlook.GetDefaultFolder(6)
print("Tipo de folder: ", outlook_folder)

### Guardar archivo de outlook

In [ ]:
# Buscar el correo más reciente con archivo Excel
mails = outlook_folder.Items

# Ordenar por fecha descendente
mails.Sort("[ReceivedTime]", True) # (mails) Es un objeto lista

''' MAIL PROPERTIES
    | (mail.Subject) (mail.ReceivedTime) (mail.SenderName)       |
    | (mail.SenderEmailAddress) (mail.To) (mail.CC)              |
    | (mail.Body) (mail.Attachments.Count) (mail.CreationTime)   |
    | (mail.LastModificationTime) (mail.EntryID)                 |
'''

# Descargar archivos Excel de los correos
def download_excel_file(mails, mail_subject, mail_sheet_name):
    for mail in mails: # Recorrer todos los correos
        if mail.Class != MAIL_ITEM_CODE: # Asegurar que el correo sea un MailItem (otros pueden ser calendario, alertas, etc)
            continue
        
        if (mail_subject.lower() in mail.Subject.lower()) & (mail.Attachments.Count > 0):
            for attachment in mail.Attachments:
                if attachment.FileName.endswith((".xlsx", ".xls")):
                    file_name = attachment.FileName
                    file_address = os.path.join(root_address+test_address, file_name)
                    attachment.SaveAsFile(file_address)

                    # Validar si hay que revisar una hoja
                    if mail_sheet_name:
                        try:
                            # Cargar solo la hoja específica
                            df = pd.read_excel(file_address, sheet_name=mail_sheet_name, header=None)
                            df = usf.fix_misplaced_headers(df)

                            # Eliminar filas que están completamente vacías
                            df_clean = df.dropna(how='all')

                            # Si la hoja esta completamente vacía, eliminar el archivo
                            if df.empty:
                                print(f"Archivo '{file_name}' con fecha {mail.ReceivedTime} está vacío, eliminando...")
                                os.remove(file_address)
                                continue
                            
                            # Si no hay datos pero si encabezados, eliminar el archivo
                            elif df_clean.shape[0] == 0:
                                print(f"Archivo '{file_name}' con fecha {mail.ReceivedTime} está vacío, eliminando...")
                                os.remove(file_address)
                                continue

                        except Exception as e:
                            print(f"Error leyendo hoja '{mail_sheet_name}' en {file_name}: {e}")
                            continue

                    # Si pasó las validaciones, devolver                    
                    return file_address, mail.ReceivedTime.strftime("%Y-%m-%d")

transportista['mail_file_address'], transportista['mail_received_time'] = download_excel_file(mails, transportista['mail_subject'], transportista['mail_sheet_name'])
ruta['mail_file_address'], ruta['mail_received_time'] = download_excel_file(mails, ruta['mail_subject'], ruta['mail_sheet_name'])

### Establecer configuracion de documentos

In [ ]:
transportista['local_file_address'] = os.path.join(root_address+test_address, transportista['local_file_name'])
transportista_resumen['local_file_address'] = os.path.join(root_address+test_address, transportista_resumen['local_file_name'])
ruta['local_file_address'] = os.path.join(root_address+test_address, ruta['local_file_name'])

# Configuracion de documentos
def imprimir_diccionario(nombre, diccionario):
    print(f'\n{nombre.upper()}:')
    for clave, valor in diccionario.items():
        print(f'{clave}: {valor}')

imprimir_diccionario('Transportista', transportista)
imprimir_diccionario('Transportista Resumen', transportista_resumen)
imprimir_diccionario('Ruta', ruta)

### Actualizar Transportista y Ruta

In [ ]:
mail_data_is_empty = False

# Verifica dos cosas
# 1. Si ambos archivos del correo son de la misma fecha
# 2. Que los archivos del correo no estan corruptos y que encontro informacion (es posible que sean de fechas antiguas y que ya esten registrados)
if transportista['mail_received_time'] == ruta['mail_received_time']:
    print(f'✅ Archivos de correo correctamente sincronizados ({transportista["mail_received_time"]})')
    print('[*] Procediendo con el analisis de actualizacion de archivos')
    transportista_updated, transportista_data_is_empty = usf.update_local_file(transportista, locaciones, root_address, test_address, vendedores)
    ruta_updated, ruta_data_is_empty = usf.update_local_file(ruta, locaciones, root_address, test_address, vendedores)
    mail_data_is_empty = transportista_data_is_empty or ruta_data_is_empty
    print("MAIL DATA IS EMPTY:", mail_data_is_empty, transportista_data_is_empty, ruta_data_is_empty)

else:
    print(f"❌ Archivos de correo no sincronizados (NO PROCEDE LA ACTUALIZACION DE ARCHIVOS)")
    print(f"Archivo de transportista: {transportista['mail_received_time']}")
    print(f"Archivo de ruta: {ruta['mail_received_time']}")

### Actualizar Transportista Resumen

In [ ]:
if mail_data_is_empty:
    print(f"❌ Archivos de correo sin informacion nueva (NO PROCEDE EL RESUMEN DE TRANSPORTISTA)")
    print(f"Archivo de transportista: {transportista['mail_received_time']}")
    print(f"Archivo de ruta: {ruta['mail_received_time']}")

else:
    print(f'✅ Archivos de correo con informacion nueva ({transportista["mail_received_time"]})')
    print('[*] Procediendo con el resumen de transportista ...')
    transportista_resumen_updated = utr.update_transportista_resumen_file(meta, transportista_updated)

### Update Files

In [ ]:
if mail_data_is_empty:
    print(f"❌ Archivos de correo sin informacion nueva (NO PROCEDE LA ESCRITURA DE ARCHIVOS)")
    print(f"Archivo de transportista: {transportista['mail_received_time']}")
    print(f"Archivo de ruta: {ruta['mail_received_time']}")    

else:
    print(f'✅ Archivos de correo con informacion nueva ({transportista["mail_received_time"]})')
    print('[*] Procediendo con la escritura de archivos ...')
    usf.save_local_file_changes(transportista_updated, transportista)
    usf.save_local_file_changes(ruta_updated, ruta)
    usf.save_local_file_changes(transportista_resumen_updated, transportista_resumen)

Continuar con envio de reporte?

In [ ]:
if transportista['mail_received_time'] == ruta['mail_received_time']:
    continue_with_automatizaion = ""
    yes = 'si'

    while continue_with_automatizaion.lower() != yes:
        continue_with_automatizaion = str(input("¿Desea continuar con la automatización? (si/no): "))

### Captura de graficos automatizados y envio de reporte

Captura de graficos

In [ ]:
# Configuracion usadas para el uso de selenium y chromedriver
METAS_Y_RESUMEN_PAGE = {
    'page_name': 'METAS Y RESUMEN',
    'page_url': 'https://app.powerbi.com/groups/me/reports/73309ec0-8d79-4111-ac74-acee01ed5375/ReportSectionf1e1f415e4cba9cc4035',
    'page_graphics': {
        1: "% CF Rechazadas",
        2: "Total CF Prog, Rech.",
        3: "Resumen de Rechazos - Meta" 
    },    
    'filter_report_by': 'month',
    'join_report_images': True,
}
DETALLES_PAGE = {
    'page_name': 'DETALLES',
    'page_url': 'https://app.powerbi.com/groups/me/reports/73309ec0-8d79-4111-ac74-acee01ed5375/ReportSection3f59cb816b6f52a24171',
    'page_graphics': {
        1: "Detalle de Cajas Físicas Rechazadas",
    },    
    'filter_report_by': 'locacion',
    'join_report_images': False,
}
RECHAZOS_PAGE = {
    'page_name': 'RECHAZOS',
    'page_url': 'https://app.powerbi.com/groups/me/reports/73309ec0-8d79-4111-ac74-acee01ed5375/ReportSection487ba5c07a522bf0fc8c',
    'page_graphics': {
        1: "% CF Rechazadas",
        2: "CF Rech. por Motivo",
        3: "CF Rech. por Transportista",
        4: "Historico % Rechazo por día"
    },
    'filter_report_by': 'locacion',
    'join_report_images': True,
}
PAGES_REPORT = [METAS_Y_RESUMEN_PAGE, DETALLES_PAGE, RECHAZOS_PAGE]

In [ ]:
if mail_data_is_empty:
    print(f"❌ Archivos de correo sin informacion nueva (NO PROCEDE LA CAPTURA DE GRAFICOS)")
    print(f"Archivo de transportista: {transportista['mail_received_time']}")
    print(f"Archivo de ruta: {ruta['mail_received_time']}")

else:
    # Automatizacion de capturas de graficos del powerbi
    print(f'✅ Archivos de correo con informacion nueva ({transportista["mail_received_time"]})')
    print('[*] Procediendo con la captura de graficos ...')
    pb_automate.powerbi_graphics_capture(locaciones, PAGES_REPORT)

Envio de reportes

In [ ]:
# if mail_data_is_empty:
#     print(f"❌ Archivos de correo no sincronizados (NO PROCEDE LA CAPTURA DE GRAFICOS)")
#     print(f"Archivo de transportista: {transportista['mail_received_time']}")
#     print(f"Archivo de ruta: {ruta['mail_received_time']}")    

# else:
#     # Automatizacion de capturas de graficos del powerbi
#     print(f'✅ Archivos de correo correctamente sincronizados ({transportista["mail_received_time"]})')
#     print('[*] Procediendo con el tratamiento de datos html')
#     sedes = smr.build_sedes_lista(locaciones)
#     smr.insert_updated_information(sedes)

calculations = smr.make_calculatiosn_for_sedes(locaciones, transportista_updated, ruta_updated, transportista, ruta)
sedes = smr.build_sedes_lista(locaciones)
smr.insert_updated_information(sedes)